In [1]:
sys.executable

'/Users/mhendrickson/anaconda/bin/python'

In [2]:
sc

In [12]:
import geopandas as gpd
import pyproj
import shapely.geometry as geom
import csv
import datetime

In [13]:
proj = pyproj.Proj(init='epsg:2263', preserve_units=True)
COLLISIONS_FN = 'data/collisions_cleaned.csv'
JANUARY_FN = 'data/1january.csv'

In [18]:
collisions = sc.textFile(COLLISIONS_FN,use_unicode=False).cache()
january = sc.textFile(JANUARY_FN,use_unicode=False).cache()

In [19]:
list(enumerate(collisons.first().split(',')))


[(0, 'DATE'),
 (1, 'TIME'),
 (2, 'BOROUGH'),
 (3, 'ZIP CODE'),
 (4, 'LATITUDE'),
 (5, 'LONGITUDE'),
 (6, 'LOCATION'),
 (7, 'ON STREET NAME'),
 (8, 'CROSS STREET NAME'),
 (9, 'OFF STREET NAME'),
 (10, 'NUMBER OF PERSONS INJURED'),
 (11, 'NUMBER OF PERSONS KILLED'),
 (12, 'NUMBER OF PEDESTRIANS INJURED'),
 (13, 'NUMBER OF PEDESTRIANS KILLED'),
 (14, 'NUMBER OF CYCLIST INJURED'),
 (15, 'NUMBER OF CYCLIST KILLED'),
 (16, 'NUMBER OF MOTORIST INJURED'),
 (17, 'NUMBER OF MOTORIST KILLED'),
 (18, 'CONTRIBUTING FACTOR VEHICLE 1'),
 (19, 'CONTRIBUTING FACTOR VEHICLE 2'),
 (20, 'CONTRIBUTING FACTOR VEHICLE 3'),
 (21, 'CONTRIBUTING FACTOR VEHICLE 4'),
 (22, 'CONTRIBUTING FACTOR VEHICLE 5'),
 (23, 'UNIQUE KEY'),
 (24, 'VEHICLE TYPE CODE 1'),
 (25, 'VEHICLE TYPE CODE 2'),
 (26, 'VEHICLE TYPE CODE 3'),
 (27, 'VEHICLE TYPE CODE 4'),
 (28, 'VEHICLE TYPE CODE 5'),
 (29, 'Counts'),
 (30, 'Collision Rate'),
 (31, 'LAT'),
 (32, 'LONG')]

# Injury Totals by Year

In [61]:
def extractInjuries(partitionId,partition):
    if partitionId == 0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if row[10]!= '0':
            injury_count = int(row[10])
            year = row[0].split('/')[2]            
            yield (year,injury_count)

injuryRecords = collisions.mapPartitionsWithIndex(extractInjuries).reduceByKey(lambda x, y: x + y)
injuryRecords.take(20)

[('2017', 10322),
 ('2016', 36552),
 ('2013', 46939),
 ('2015', 43263),
 ('2012', 23564),
 ('2014', 43312)]

# Death Totals by Year

In [83]:
def extractDeaths(partitionId,partition):
    if partitionId == 0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if row[11]!= '0':
            death_count = int(row[11])
            year = row[0].split('/')[2]            
            yield (year,death_count)

deathRecords = collisions.mapPartitionsWithIndex(extractDeaths).reduceByKey(lambda x, y: x + y)
deathRecords.take(20)

[('2017', 33),
 ('2016', 128),
 ('2013', 243),
 ('2015', 201),
 ('2012', 122),
 ('2014', 215)]

# Contributing Factors

In [103]:
def accidentFactors(partitionId,partition):
    if partitionId == 0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if row[18]!= 'Unspecified' and row[18] != '':
            cause = row[18]
            year = row[0].split('/')[2]            
            yield ((cause,year),1)

accidentFactors = collisions.mapPartitionsWithIndex(accidentFactors).reduceByKey(lambda x,y: x+y) #.max(lambda x:x[1])
sorted(accidentFactors.collect())

[(('Accelerator Defective', '2012'), 45),
 (('Accelerator Defective', '2013'), 64),
 (('Accelerator Defective', '2014'), 77),
 (('Accelerator Defective', '2015'), 66),
 (('Accelerator Defective', '2016'), 65),
 (('Accelerator Defective', '2017'), 23),
 (('Aggressive Driving/Road Rage', '2012'), 304),
 (('Aggressive Driving/Road Rage', '2013'), 463),
 (('Aggressive Driving/Road Rage', '2014'), 505),
 (('Aggressive Driving/Road Rage', '2015'), 689),
 (('Aggressive Driving/Road Rage', '2016'), 383),
 (('Aggressive Driving/Road Rage', '2017'), 189),
 (('Alcohol Involvement', '2012'), 630),
 (('Alcohol Involvement', '2013'), 1273),
 (('Alcohol Involvement', '2014'), 1490),
 (('Alcohol Involvement', '2015'), 1530),
 (('Alcohol Involvement', '2016'), 1029),
 (('Alcohol Involvement', '2017'), 468),
 (('Animals Action', '2012'), 34),
 (('Animals Action', '2013'), 69),
 (('Animals Action', '2014'), 83),
 (('Animals Action', '2015'), 88),
 (('Animals Action', '2016'), 111),
 (('Animals Action', '